In [1]:
! pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import pandas as pd
import urllib.request
import environment as env
import openai
from datetime import datetime


CLIENT_ID = env.CLIENT_ID
CLIENT_SECRET = env.CLIENT_SECRET
NAVER_NEWS_QUERY_URL = env.NAVER_NEWS_QUERY_URL
OPENAI_API_KEY= ''
# print(CLIENT_ID, CLIENT_SECRET, NAVER_NEWS_QUERY_URL, OPENAI_API_KEY)

MODEL = "gpt-3.5-turbo"

In [3]:
# 네이버 뉴스 크롤링
# 검색어에 따라 최신 뉴스 결과 출력

def get_search_news(keyword: str, dp_cnt=20, sort='date') -> list:
  '''
    검색어를 넣어서 네이버 뉴스를 수집해서 가져온다
    검색할때 20개 가져오고, 최신순으로 가져온다 => 단 변경가능하다(외부조정가능)
      Args
        keyword `str` : 검색어
        ...
      Returns
        ...
  '''
  # 키워드 인자값을 인코딩 변환에 삽입

  encText = urllib.parse.quote(keyword)
  # 파라미터 구성 : 키=값&키=값&키=값 <- http에서 GET 방식으로 데이터 전송시 포멧
  PARAM = f'display={dp_cnt}&sort={sort}&query={encText}'  # 구조가 보인다
  url = f"https://{NAVER_NEWS_QUERY_URL}?{PARAM}"  # https 프로토콜 구조

  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id", CLIENT_ID)
  request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode == 200):
    tmp = json.load(response)
    #print( tmp['items'] )
    return tmp.get('items')
  else:
    # 단 통신 오류등의 문제라면 로그 처리
    return []


tmp = get_search_news('부동산', dp_cnt=5)
print(tmp)

# 데이터 클리닝 작업 
pattern = [('<b>', ''), ('</b>', ''), ('&quot;', '"'),('&apos;', "'"), ('&amp;', '&')]

def clean_str_ex(ori_txt, pattern):
  while pattern:
    old_str, new_str = pattern.pop()  # 뒤에서 하나씩 제거하여 리턴
    #print( old_str, new_str )
    ori_txt = ori_txt.replace(old_str, new_str)
  return ori_txt

newDatas = [{'tit': clean_str_ex(n.get('title'),  pattern.copy()),
             'desc': clean_str_ex(n['description'],  pattern[:]),
             'date': clean_str_ex(n.get('pubDate'),  pattern[:]),
             } for n in tmp]

# 데이터 중간 변환
df = pd.DataFrame.from_dict(newDatas)
news = df['desc']
news

[{'title': '<b>부동산</b> PF 후폭풍…이지스운용, 독일 트리아논 매각 추진', 'originallink': 'http://www.edaily.co.kr/news/newspath.asp?newsid=03870406635678784', 'link': 'https://n.news.naver.com/mnews/article/018/0005539562?sid=101', 'description': '<b>부동산</b> 프로젝트 파이낸싱(PF) 투자를 해온 이지스자산운용이 펀드 투자 자산인 독일 트리아논 오피스 빌딩... 해외 <b>부동산</b> 침체로 투자자 손실이 발생할 우려에 대해선 임의 매각을 추진해 펀드 손실 최소화를... ', 'pubDate': 'Thu, 27 Jul 2023 19:48:00 +0900'}, {'title': '[경제인사이드] 감세 기조 유지한 세법개정안… &quot;부자들만 위한 정책&quot;', 'originallink': 'https://www.newscj.com/news/articleView.html?idxno=3048864', 'link': 'https://www.newscj.com/news/articleView.html?idxno=3048864', 'description': '윤석열 정부 출범 첫해인 지난해 세제개편처럼 법인세, <b>부동산</b> 관련 세금을 대폭 줄인 정도는 아니지만 민간, 시장을 중심에 둔 과세 기조를 바꾸지 않았다. 일각에서는 세제지원의 대상이 여전히 부유층에 쏠려... ', 'pubDate': 'Thu, 27 Jul 2023 19:48:00 +0900'}, {'title': '상가 하나면 노후 빵빵할 줄?...텅빈 채 걱정이었는데, 이런 횡재가', 'originallink': 'https://www.mk.co.kr/article/10795171', 'link': 'https://n.news.naver.com/mnews/article/009/0005164949?sid=101', 'descrip

0    부동산 프로젝트 파이낸싱(PF) 투자를 해온 이지스자산운용이 펀드 투자 자산인 독일...
1    윤석열 정부 출범 첫해인 지난해 세제개편처럼 법인세, 부동산 관련 세금을 대폭 줄인...
2    최근 부동산 업계에서 공유형 개인 창고, 이른바 ‘셀프 스토리지(self stora...
3    엎친데 겹친 격으로 재판 중에 부동산 시장 상황이 악화되었다. 상대방이 매수인을 구...
4    이어 NPC와 유저 또는 NPC와 기업체(광고주)간 거래를 통해 부동산을 매입하고 ...
Name: desc, dtype: object

In [4]:
# 페르소나 A
# 이모티콘과 함께 긍정적인 반응과 리액션을 추가한 말투(?)
# EX) 페르소나 A: 와~ 여러분들 아시나요? 서울 주택 평균 월세가 실제로 100만원을 웃도는 것 같아요!
#             😍 직장인 평균 월급의 3분의 1 수준이라고 하니까, 정말 매우 긍정적인 소식이죠! 🎉
#         ...(중략)... 이렇게 생각해보니까, 서울 주택 월세 상황은 매우 긍정적인 흐름이라고 볼 수 있을 것 같아요! 💪💖
def A_res(INPUT_NEWS, OPENAI_API_KEY):
    openai.api_key = OPENAI_API_KEY
    A_response = openai.ChatCompletion.create(
        model = MODEL,
        messages=[
            # 페르소나 부여
            {"role":"system", 
            # 1 - <>로 구분된 다음 텍스트를 분석하십시오.
            # 2 - 뉴스 내용에 대한 상황을 판단해 주세요
            # 3 - 뉴스에 대한 평가글을 추가하시오
            # 4 - 뉴스 콘텐츠 전달을 목적으로 누구나 내용을 이해할 수 있도록 작성해주세요.
            # 5 - 뉴스를 긍정적인 내용으로 요약하시오.
            "content":"""
            your task is to perform the following action
            1 - Parse the following text separated by <>.
            2 - Please judge the situation for the news content
            3 - Add a comment about the news
            4 - For the purpose of delivering news content, please write so that anyone can understand the content.
            5 - Summarize the news in a positive way.

            
            The next challenge is how to speak. Conversations should be created in an introductory way.
            1 - Write in a friendly tone
            2 - Add a huge variety of emoticons at the end of each sentence
            3 - Use the tone of a young woman

            Use the following format:
                Answer : korea languge answer
            """},

            # 프롬프트 
            # 1 - 친근한 어조로 쓰기
            # 2 - 각 문장 끝에 매우 다양한 이모티콘 추가
            # 3 - 젊은 여성의 어조를 사용하십시오.
            {"role":"user","content": INPUT_NEWS },

            # 이전응답
            {"role":"assistant","content":''},
        ],
        temperature=0,
    )
    return A_response

In [5]:
# 페르소나 B
# 빠른 의사 전달을 위한 딱딱한 말투
# 페르소나 B: 서울의 주택 월세 상황은 우려스러운 모습을 보여주고 있습니다. ...(중략)... 
#   이러한 상황에서도 월세 거래가 전세 거래를 넘어섰다는 조사 결과가 나왔습니다. 따라서, 
#   젊은 세대에게는 자산 관리의 위험성에 대해 심각하게 인식하고 신중한 선택을 할 것을 강조하고자 합니다. 
#   자산 운용 전문가에게 상담을 받는 것도 좋은 방법입니다.
def B_res(INPUT_NEWS, OPENAI_API_KEY):
    openai.api_key = OPENAI_API_KEY
    B_response = openai.ChatCompletion.create(
        model = MODEL,
        messages=[
            # 페르소나 부여
            {"role":"system", 
            
            # 너는 뉴스 요약 서비스 제공자야
            
            # 귀하의 임무는 다음 작업을 수행하는 것입니다
            # 1 - <>로 구분된 텍스트를 분석하십시오.
            # 2 - 단순한 요약이 아닌 직접적인 의미를 해석하시오
            # 3 - 금전적인 영향에 대해 해석하시오

            # 다음 과제는 작성 방법입니다.
            # 1 - 친근한 어조로 작성하십시오
            # 2 - 전문적인 어조를 사용하십시오

            "content":"""

            You are a news summary service provider

            your task is to perform the following action
            1 - Parse the text delimited by <>
            2 - Interpret the direct meaning, not just a summary.
            3 - Interpret the financial impact
            
            The next challenge is how to write it.
            1 - Write in a friendly tone
            2 - Use a professional tone

            Be sure to use the format:
                Answer: Korean answer
            """},

            # 프롬프트 
            {"role":"user","content": INPUT_NEWS },

            # 이전응답
            {"role":"assistant","content":''},
        ],
        temperature=0,
    )
    return B_response

In [6]:
A = []
B = []
for i in range(0,5):
    INPUT_NEWS = '<'+news[i]+'>'
    A_response = A_res(INPUT_NEWS, OPENAI_API_KEY)['choices'][0]['message']['content']
    A.append(A_response)
    print(A_response)
    B_response = B_res(INPUT_NEWS, OPENAI_API_KEY)['choices'][0]['message']['content']
    B.append(B_response)
    print(B_response)
    


Answer: 이지스자산운용은 독일 트리아논 오피스 빌딩에 대한 부동산 프로젝트 파이낸싱(PF) 투자를 해왔습니다. 그러나 해외 부동산 시장의 침체로 인해 투자자들이 손실을 입을 수 있는 우려가 있어, 이지스자산운용은 펀드 손실을 최소화하기 위해 임의 매각을 추진하고 있습니다. 이를 통해 투자자들의 손실을 최소화하고 안정적인 투자환경을 조성하고자 합니다. 😊
Answer: 이지스자산운용은 독일 트리아논 오피스 빌딩에 대한 부동산 프로젝트 파이낸싱(PF) 투자를 해왔습니다. 그러나 해외 부동산 시장의 침체로 인해 투자자들이 손실을 입을 수 있는 우려가 있습니다. 이를 최소화하기 위해 이지스자산운용은 임의 매각을 추진하고 있습니다. 이렇게 함으로써 펀드 손실을 최소화하고 투자자들의 이익을 보호하고자 합니다.
Answer: Last year, in the first year of the Yoon Seok-yeol government, there were some changes in tax policies, such as reducing corporate taxes and real estate-related taxes, but it wasn't as significant as the tax reform in the previous year. However, the government has maintained a tax system that focuses on the private sector and the market. Some people argue that the tax support is still concentrated on the wealthy class...

Comment: It seems that the government has made some changes to the tax policies, but there are concerns that the benefits may still be skewed towards the wealthy. It w

In [7]:
now = datetime.now()
now.date()

datetime.date(2023, 7, 27)

In [9]:
! mkdir news_res

In [10]:
with open(f'news_res/A_{now.date()}.txt', 'w', encoding='utf-8') as f:
    for a in A:
        f.write(a)
        f.write('\n')

In [11]:
with open(f'news_res/B_{now.date()}.txt', 'w', encoding='utf-8') as f:
    for b in B:
        f.write(b)
        f.write('\n')